In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
archive = 'data/dataset.csv'

df = pd.read_csv(archive)
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,LABEL_TARGET
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,False
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,True
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,False
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,False
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,False


## Análise Exploratória e Limpeza de Dados

In [4]:
df.LABEL_TARGET.value_counts()

LABEL_TARGET
False    9200
True     2300
Name: count, dtype: int64

In [5]:
df["LABEL_TARGET"] = df["LABEL_TARGET"].astype(int)

In [8]:
print(f"Número de colunas: {len(df.columns)}")

Número de colunas: 179


In [9]:
# Verificando se há valores ausentes
df.isnull().values.any()

False

In [11]:
# Extrai a lista de colunas
lista_de_colunas = df.columns.tolist()

In [12]:
# Colunas das variáveis de entrada (input)
colunas_entrada = lista_de_colunas[0:178]

In [13]:
# Checando se temos colunas duplicadas nos dados de entrada
dup_cols = set([x for x in colunas_entrada if colunas_entrada.count(x) > 1])
print(dup_cols)
assert len(dup_cols) == 0, "você duplicou colunas em cols_input"

set()


In [14]:
# Checando se temos colunas duplicadas no dataset completo
dup_cols = set([x for x in lista_de_colunas if lista_de_colunas.count(x) > 1])
print(dup_cols)
assert len(dup_cols) == 0,'você duplicou colunas em lista_de_colunas'

set()


A prevalência é a porcentagem das amostras que têm a característica que você está tentando prever. Em nosso caso, significa que as pessoas que renovaram o seguro são da classe positiva (ocorrência do evento) e quem não comprou é da classe negativa (não ocorreu o evento).

Os termos positivo e negativo não têm conotação de coisa boa ou ruim. É apenas a nomenclatura usada para indicar a ocorrência ou não do evento.

A taxa é calculada por (número de amostras positivas / número de amostras). Portanto, uma taxa de prevalência de 0,2 significa que 20% de nossa amostra renovou o seguro do carro.

In [15]:
# Esta função calcula a prevalência da classe positiva (label = 1)
def calcula_prevalencia(y_actual):
    return sum(y_actual) / len(y_actual)

print("Prevalência da classe positiva: %.3f"% calcula_prevalencia(df["LABEL_TARGET"].values))

Prevalência da classe positiva: 0.200


> O desbalanceamento de classe é um problema que terá que ser resolvido durante o pré-processamento dos dados.

## Divisão dos Dados Mantendo a Prevalência de Classe

In [22]:
# Gerando amostras aleatórias dos dados
df_data = df.sample(n = len(df)).reset_index(drop = True)

# Extrai uma amostra de 30% dos dados de forma aleatória
df_amostra_30 = df_data.sample(frac = 0.3)

print(f"Tamanho da divisão de validação / teste: {(len(df_amostra_30) / len(df_data))}")

Tamanho da divisão de validação / teste: 0.3


In [23]:
# Fazendo a divisão

# Dados de teste
df_teste = df_amostra_30.sample(frac = 0.5)

# Dados se validação
df_valid = df_amostra_30.drop(df_teste.index)

# Dados de treino
df_treino = df_data.drop(df_amostra_30.index)

In [24]:
# Verifique a prevalência de cada subconjunto
print(
    "Teste(n = %d): %.3f"
    % (len(df_teste), calcula_prevalencia(df_teste.LABEL_TARGET.values))
)
print(
    "Validação(n = %d): %.3f"
    % (len(df_valid), calcula_prevalencia(df_valid.LABEL_TARGET.values))
)
print(
    "Treino(n = %d): %.3f"
    % (len(df_treino), calcula_prevalencia(df_treino.LABEL_TARGET.values))
)

Teste(n = 1725): 0.197
Validação(n = 1725): 0.208
Treino(n = 8050): 0.199


## Balanceamento de Classe

Vamos aplicar a estratégia de undersampling. Leia o material no Capítulo 8 com a definição dos conceitos.

In [27]:
# Cria um índice com True/False
indice = df_treino.LABEL_TARGET == 1

# Define valores positivos e negativos do índice
df_train_pos = df_treino.loc[indice]
df_train_neg = df_treino.loc[~indice]

# Valor mínimo de registros entre classe positiva e negativa
valor_minimo = np.min([len(df_train_pos), len(df_train_neg)])

In [28]:
# Obtém valores aleatórios para o dataset de treino
df_treino_final = pd.concat([df_train_pos.sample(n = valor_minimo, random_state = 69), 
                             df_train_neg.sample(n = valor_minimo, random_state = 69)], 
                            axis = 0, 
                            ignore_index = True)

In [29]:
# Amostragem aleatória do dataset de treino
df_treino_final = df_treino_final.sample(n = len(df_treino_final), random_state = 69).reset_index(drop = True)

In [30]:
# Balanceamento do conjunto de dados
print('Balanceamento em Treino(n = %d): %.3f'%(len(df_treino_final), 
                                               calcula_prevalencia(df_treino_final.LABEL_TARGET.values)))

Balanceamento em Treino(n = 3204): 0.500


In [31]:
## Salvando o Resultado do Pré-Processamento

df_treino.to_csv('data/dados_treino.csv', index = False)
df_treino_final.to_csv('data/dados_treino_final.csv', index = False)
df_valid.to_csv('data/dados_valid.csv', index = False)
df_teste.to_csv('data/dados_teste.csv', index = False)

In [32]:
# Salvamos os nomes dos dados de entrada (colunas preditoras) para facilitar a utilização mais tarde
pickle.dump(colunas_entrada, open('data/colunas_entrada.sav', 'wb'))